In [1]:
import json, torch
import lightning  as L
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from torch.nn import functional as F
import csv
from collections import defaultdict
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle
import pandas as pd
import pprint

/home/davidteng/anaconda3/envs/wsd/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### load cwb data

In [2]:
from collections import defaultdict
import numpy as np
import csv
from scipy.interpolate import interp1d

# 定义月份和特征
months = ['01','02','03','04','05','06','07','08', '09', '10']
features = ['rain', 'raintime', 'solarpower', 'suntime', 'temp', 'uv']

# 初始化 cwb_data_dict，四层嵌套：特征 -> 月份 -> 天数 -> 时间索引
# 时间索引初始为小时（0-23），插值后为分钟（0-1439）
cwb_data_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))

# 读取气象数据
for month in months:
    for feature in features:
        csv_path = f'../cwbdata/{month}/{feature}-{month}.csv'
        try:
            with open(csv_path, 'r', encoding='utf-8') as csv_file:
                reader = csv.reader(csv_file)
                for row in reader:
                    if row[0].isdigit():
                        day = int(row[0])
                        for hour_index, value in enumerate(row[1:], start=0):
                            if hour_index < 24:
                                try:
                                    cwb_data_dict[feature][int(month)][day][hour_index] = float(value)
                                except (ValueError, TypeError):
                                    cwb_data_dict[feature][int(month)][day][hour_index] = None  # 标记为无效值
        except FileNotFoundError:
            print(f"文件未找到: {csv_path}")

# 初始化插值后的数据字典
cwb_data_interp_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))

def interpolate_minute_data(hourly_values):
    """
    对一天的24小时数据进行每分钟插值，返回1440分钟的浮点数列表。
    使用三次样条插值，并确保插值后的值不为负。
    """
    # 已知的分钟位置（每小时的第30分钟）
    known_minutes = np.array([h * 60 + 30 for h in range(24)])
    known_values = np.array(hourly_values)
    
    # 创建所有分钟的索引
    all_minutes = np.arange(1440)
    
    # 处理缺失值后进行插值
    # 使用三次样条插值
    try:
        # 创建插值函数
        spline = interp1d(known_minutes, known_values, kind='cubic', fill_value="extrapolate")
        interpolated = spline(all_minutes)
    except Exception as e:
        print(f"插值失败: {e}")
        interpolated = np.zeros(1440)
    
    # 确保插值后的值不为负
    interpolated = np.clip(interpolated, a_min=0, a_max=None)
    
    return interpolated.tolist()

# 处理每个特征、每个月、每天的数据
for feature in features:
    for month in range(1, 11):  # months '01' to '10' correspond to 1 to 10
        for day in cwb_data_dict[feature][month]:
            # 获取24小时的值
            hourly_values = [cwb_data_dict[feature][month][day].get(hour, None) for hour in range(24)]
            
            # 填充 None 值
            for hour in range(24):
                if hourly_values[hour] is None:
                    # 查找前一个有效值
                    prev = None
                    for h in range(hour - 1, -1, -1):
                        if hourly_values[h] is not None:
                            prev = hourly_values[h]
                            break
                    # 查找后一个有效值
                    next_val = None
                    for h in range(hour + 1, 24):
                        if hourly_values[h] is not None:
                            next_val = hourly_values[h]
                            break
                    # 计算平均值
                    if prev is not None and next_val is not None:
                        hourly_values[hour] = (prev + next_val) / 2
                    elif prev is not None:
                        hourly_values[hour] = prev
                    elif next_val is not None:
                        hourly_values[hour] = next_val
                    else:
                        hourly_values[hour] = 0.0  # 无有效值，填充0.0
            
            # 进行插值
            interpolated_minutes = interpolate_minute_data(hourly_values)
            
            # 将1440分钟的数据分成24个小时，每小时60分钟
            for hour in range(24):
                start_min = hour * 60
                end_min = start_min + 60
                cwb_data_interp_dict[feature][month][day][hour] = interpolated_minutes[start_min:end_min]
                # cwb_data_interp_dict[feature][month][day][hour] = [cwb_data_dict[feature][month][day][hour]] * 60

# 确保插值后的数据没有 None
for feature in features:
    for month in range(1, 11):
        for day in cwb_data_interp_dict[feature][month]:
            for hour in range(24):
                minute_data = cwb_data_interp_dict[feature][month][day][hour]
                if any([x is None for x in minute_data]):
                    # 进一步处理可能的 None（理论上已处理完毕）
                    cwb_data_interp_dict[feature][month][day][hour] = [0.0 if x is None else x for x in minute_data]

# 打印一个示例
print(cwb_data_interp_dict['rain'][1][1][1])  # 1月1日的1点的rain数据，60个浮点数


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [3]:
class Cwb2LocModel(L.LightningModule):
    def __init__(self, input_dim=7, hidden_dim=128, num_layers=2, output_dim=1, learning_rate=1e-3, delta=1.0):
        super(Cwb2LocModel, self).__init__()
        self.save_hyperparameters()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.delta = delta  # 動態設置 delta

        # 定義 LSTM 層
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.15, bidirectional=True)

        # 定義全連接層
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim // 4)
        self.fc2 = nn.Linear(hidden_dim // 4, input_dim)
        self.fc3 = nn.Linear(input_dim, 1)

        # 定義損失函數（Huber Loss）
        # self.criterion = nn.HuberLoss(delta=self.delta)
        # self.criterion = nn.MSELoss()
        self.criterion = nn.L1Loss()
        # self.criterion = log_cosh_loss

    def log_cosh_loss(y_pred, y_true):
        loss = torch.mean(torch.log(torch.cosh(y_pred - y_true)))
        return loss

    def forward(self, x):
        # 初始化隱藏狀態和細胞狀態
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(self.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(self.device)
        residual = x
        
        # 前向傳播 LSTM
        out, _ = self.lstm(x, (h0, c0))

        # 通過全連接層得到最終輸出
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out += residual
        out = self.fc3(out)
        
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)  # 使用 Huber Loss
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.criterion(y_hat, y)  # 使用 Huber Loss
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.AdamW(
            self.parameters(),
            lr=self.hparams.learning_rate,  # 初始学习率
            weight_decay=1e-2               # 权重衰减
        )
        
        # 定义调度器
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',             # 目标是最小化验证损失
            factor=0.7,             # 学习率每次调整的倍率
            patience=10,            # 验证损失未改善的 epoch 数
            threshold=1e-4,         # 改善的阈值
            cooldown=5,             # 调整后等待的冷却时间
            min_lr=1e-5,            # 学习率的下限
            verbose=True            # 打印学习率变化信息
        )
        
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",  # 监控的指标
                "frequency": 1          # 每个 epoch 检查一次
            }
        }
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.9)
        return [optimizer], [scheduler]



In [4]:
import math

loc_detail = {
    1: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(181)), 'cos': math.cos(math.radians(181))},
        'coordinate': (23.899444, 121.544444)  # 转换为十进制
    },
    2: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(175)), 'cos': math.cos(math.radians(175))},
        'coordinate': (23.899444, 121.544722)
    },
    3: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(180)), 'cos': math.cos(math.radians(180))},
        'coordinate': (23.899722, 121.544722)
    },
    4: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(161)), 'cos': math.cos(math.radians(161))},
        'coordinate': (23.899444, 121.544444)
    },
    5: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(208)), 'cos': math.cos(math.radians(208))},
        'coordinate': (23.899444, 121.544722)
    },
    6: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(208)), 'cos': math.cos(math.radians(208))},
        'coordinate': (23.899444, 121.544444)
    },
    7: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(172)), 'cos': math.cos(math.radians(172))},
        'coordinate': (23.899444, 121.544444)
    },
    8: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(219)), 'cos': math.cos(math.radians(219))},
        'coordinate': (23.899722, 121.544722)
    },
    9: {
        'floor': 3,
        'face': {'sin': math.sin(math.radians(151)), 'cos': math.cos(math.radians(151))},
        'coordinate': (23.899444, 121.544444)
    },
    10: {
        'floor': 3,
        'face': {'sin': math.sin(math.radians(223)), 'cos': math.cos(math.radians(223))},
        'coordinate': (23.899444, 121.544444)
    },
    11: {
        'floor': 1,
        'face': {'sin': math.sin(math.radians(131)), 'cos': math.cos(math.radians(131))},
        'coordinate': (23.899722, 121.544722)
    },
    12: {
        'floor': 1,
        'face': {'sin': math.sin(math.radians(298)), 'cos': math.cos(math.radians(298))},
        'coordinate': (23.899722, 121.544722)
    },
    13: {
        'floor': 1,
        'face': {'sin': math.sin(math.radians(249)), 'cos': math.cos(math.radians(249))},
        'coordinate': (23.898889, 121.539444)
    },
    14: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(197)), 'cos': math.cos(math.radians(197))},
        'coordinate': (23.898889, 121.539444)
    },
    15: {
        'floor': 5,
        'face': {'sin': math.sin(math.radians(127)), 'cos': math.cos(math.radians(127))},
        'coordinate': (24.009167, 121.617222)
    },
    16: {
        'floor': 1,
        'face': {'sin': math.sin(math.radians(82)), 'cos': math.cos(math.radians(82))},
        'coordinate': (24.008889, 121.617222)
    },
    # 17: {
    #     'floor': 1,
    #     'face': None,  # 没有提供具体朝向信息
    #     'coordinate': None  # 自家仓库无具体坐标
    # }
    17: {
        'floor': 1,
        'face': {'sin': math.sin(math.radians(270)), 'cos': math.cos(math.radians(270))},
        'coordinate': (23.955563853486503, 121.59588278748417)
    }
}


In [5]:
model_path = f'./saved_models_v3_2/best-checkpoint-g.ckpt'
model = Cwb2LocModel.load_from_checkpoint(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)  # 將模型移動到 GPU（如果可用）
model.eval()  # 切換模型到評估模式

Cwb2LocModel(
  (lstm): LSTM(11, 128, num_layers=2, batch_first=True, dropout=0.15, bidirectional=True)
  (fc1): Linear(in_features=256, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=11, bias=True)
  (fc3): Linear(in_features=11, out_features=1, bias=True)
  (criterion): L1Loss()
)

In [6]:
df = pd.read_csv('./up.csv')
pred_date_list = []
# pred_start_time = [9, 0]
# pred_end_time = [16, 59]
for seq_id in df['序號']:
    seq_id = str(seq_id)
    new = (int(seq_id[4:6]), int(seq_id[6:8]), int(seq_id[-2:]))
    if new not in pred_date_list:
        pred_date_list.append(new)
# for s in pred_date_list:
#     print(s)


In [7]:
max_len = 65
start_time = [6, 30]
end_time = [17, 19]

In [8]:
def extract_features(cwb_data_interp_dict, features, location, month, day, hour, minute):
    try:
        newx = [cwb_data_interp_dict[feature][month][day][hour][minute] for feature in features]
        # newx.append(int(location))  # 添加 location 作为特征
        newx.append(float(loc_detail[location]['floor']))
        newx.append(float(loc_detail[location]['face']['sin']))
        newx.append(float(loc_detail[location]['face']['cos']))
        newx.append(float(loc_detail[location]['coordinate'][0]))
        newx.append(float(loc_detail[location]['coordinate'][1]))
        # newx.append(int(hour))
        # newx.append(int(minute))
    except Exception as e:
        print(f"Error at {location}-{month}-{day} {hour}:{minute}: {e}")
        newx = [0.0] * (len(features) + 1)
    return newx
def collate_date_data(month, day, location, features):
    data = []
    for hour in range(start_time[0], end_time[0] + 1):
        minute_start = start_time[1] if hour == start_time[0] else 0
        minute_end = end_time[1]+1 if hour == end_time[0] else 60

        for start_minute in range(minute_start, minute_end, 10):  # 每10分钟一组
            group_x = []

            for minute in range(start_minute, start_minute + 10):
                if minute >= minute_end:
                    break
                # 提取特征数据
                newx = extract_features(cwb_data_interp_dict, features, location, month, day, hour, minute)
                # 提取目标数据
                group_x.append(newx)

                # 计算该时间段的平均值
            averaged_x = np.mean(group_x, axis=0).tolist()
            data.append(averaged_x)
    return data

In [9]:
pred_date_data = []
for month, day, loc in pred_date_list:
    # print(month, day, loc)
    pred_date_data.append(collate_date_data(month, day, loc, features))
for _i, data in enumerate(pred_date_data):
    if len(data) != 65:
        print(_i)
    for _j, features in enumerate(data):
        if len(features) != 11:
            print(_j)

In [10]:
print(pred_date_data[0])

[[0.0, 8.673617379884036e-20, 0.015478052584736978, 1.6940658945086008e-22, 16.923125800433947, 6.938893903907229e-19, 5.0, -0.017452406437283633, -0.9998476951563913, 23.899443999999995, 121.54444400000003], [0.0, 0.0, 0.031109410399882204, 0.0, 16.975148869162926, 0.0, 5.0, -0.017452406437283633, -0.9998476951563913, 23.899443999999995, 121.54444400000003], [0.0, 0.0, 0.05277921493486255, 0.0, 17.033501647059474, 0.0, 5.0, -0.017452406437283633, -0.9998476951563913, 23.899443999999995, 121.54444400000003], [0.0, 0.0, 0.08128552490994251, 0.0, 17.105537487389963, 0.0, 5.0, -0.017452406437283633, -0.9998476951563913, 23.899443999999995, 121.54444400000003], [0.0, 0.0, 0.11742639904538654, 0.0, 17.198609743420786, 0.0, 5.0, -0.017452406437283633, -0.9998476951563913, 23.899443999999995, 121.54444400000003], [0.0, 0.0, 0.16199989606145912, 0.0, 17.320071768418316, 0.0, 5.0, -0.017452406437283633, -0.9998476951563913, 23.899443999999995, 121.54444400000003], [0.00430594478335561, 0.004071

In [11]:
# 加載標準化器
with open(f'./scalar_v3_2/x_scaler-b2.pkl', 'rb') as f:
    x_scaler = pickle.load(f)
with open(f'./scalar_v3_2/y_scaler-b2.pkl', 'rb') as f:
    y_scaler = pickle.load(f)

In [12]:
def get_single_day_output(x, x_scaler, y_scaler, device):
    """
    输入一天的数据特征，进行标准化后传入模型进行预测，并进行逆标准化，返回预测结果。
    """
    # 转为 NumPy 数组并标准化
    x = np.array(x, dtype=np.float32)
    x = x_scaler.transform(x.reshape(-1, x.shape[-1])).reshape(x.shape)

    # 转为 PyTorch 张量，移动到指定设备
    x_tensor = torch.tensor(x, dtype=torch.float32).to(device).unsqueeze(0)  # [batch=1, seq_len, features]
    print(x)
    # 确保模型处于评估模式
    with torch.no_grad():
        # 模型预测
        y_pred = model(x_tensor)  # [batch=1, seq_len, 1]
        y_pred = y_pred.squeeze(0).cpu().numpy()  # [seq_len, 1]

    # 逆标准化
    y_pred = y_scaler.inverse_transform(y_pred)

    return y_pred.flatten().tolist()[15:63]


In [13]:
get_single_day_output(pred_date_data[1], x_scaler, y_scaler, device)

[[-0.08735863 -0.3578402  -1.2380649  -0.9025123  -1.6747721  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.2134247  -0.9025123  -1.6520964  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.18135    -0.9025123  -1.6255478  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.142147   -0.9025123  -1.5944269  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.0961215  -0.9025123  -1.5580332  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.0435797  -0.9025123  -1.515667   -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -0.9848605  -0.901821   -1.4667047  -1.02878
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -0.9207229  -0.9000755  

[203.81906127929688,
 208.18650817871094,
 223.05645751953125,
 247.7724609375,
 303.0348205566406,
 386.715087890625,
 483.6344909667969,
 568.3871459960938,
 726.9251098632812,
 946.3670654296875,
 1206.1376953125,
 1412.639404296875,
 1498.865234375,
 1504.384521484375,
 1450.355224609375,
 1408.7164306640625,
 1439.2109375,
 1642.95654296875,
 1825.7515869140625,
 1551.6876220703125,
 1188.75927734375,
 964.5963134765625,
 1022.6766967773438,
 1165.6480712890625,
 1341.45361328125,
 1208.1201171875,
 940.88330078125,
 724.2374877929688,
 614.615234375,
 652.9640502929688,
 786.8558349609375,
 904.1058959960938,
 935.2508544921875,
 806.3057861328125,
 587.8196411132812,
 356.0961608886719,
 215.593994140625,
 156.52711486816406,
 126.44869995117188,
 105.34422302246094,
 76.78857421875,
 49.01875305175781,
 37.77574157714844,
 29.360366821289062,
 21.971633911132812,
 16.046554565429688,
 12.962844848632812,
 11.103775024414062]

In [14]:
# 存储预测结果
predictions = []

# 遍历所有需要预测的数据
for i, x in enumerate(pred_date_data):
    try:
        # 获取单天预测结果（返回 list）
        y_pred = get_single_day_output(x, x_scaler, y_scaler, device)
        y_pred = [max(0, val) for val in y_pred]
        for y in y_pred:
            predictions.append(y)
    except Exception as e:
        print(f"Error in prediction for index {i}: {e}")
        predictions.append([0.0] * max_len)  # 用全零填补


[[-0.08735863 -0.3578402  -1.2509019  -0.9025123  -1.9163349  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.2373137  -0.9025123  -1.9051118  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.2184765  -0.9025123  -1.8925229  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.1936965  -0.9025123  -1.8769827  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.1622798  -0.9025123  -1.8569038  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08735863 -0.3578402  -1.1235328  -0.9025123  -1.8307004  -1.0414684
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.08367631 -0.34048244 -1.0768416  -0.9024899  -1.7969139  -1.0312258
   0.8580207   0.09847348 -0.732896   -0.37607345 -0.38508055]
 [-0.07437794 -0.2966358  -1.0226182  -0.9024333

In [15]:
print((predictions[1]))

92.35725402832031


In [16]:
# 平均每段时间的预测值作为答案
submission_data = []
for seq_id, y in zip(df['序號'], predictions):
    # 获取对应时间段的平均值
    # seq_id = str(seq_id)
    # time_index = int(seq_id[8:10])  # 提取时间索引
    # pred_value = np.mean(y_pred[time_index])  # 平均值
    submission_data.append([seq_id, y])

# 转为 DataFrame
submission_df = pd.DataFrame(submission_data, columns=['序號', '答案'])

# 保存为 CSV 文件
submission_df.to_csv('./upC_2-g2.csv', index=False)
print("提交文件已生成: ./submission.csv")


提交文件已生成: ./submission.csv
